# METOD algorithm procedure guide for minimum of several quadratic forms

# 1. Import libraries

The following libraries are required in order for the METOD Algorithm to run.

In [1]:
import numpy as np
import sys  
sys.path.insert(0, 'src')
import metod_testing as mtv3
from numpy import linalg as LA
import pandas as pd

/Users/megscammell/anaconda3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


# 2. Define function and gradient

Minimum of several quadratic forms:
 \begin{equation}
 \label{eq:model}
f(x_n^{(k)})=\min_{1\le p \le P}  (x_n^{(k)}-x_{0p})^T A_p^T \Sigma_p A_p (x_n^{(k)}-x_{0p}),
\end{equation}
where $x_n^{(k)}$ ($n=1,...,N$) is a point with $k$ iterations of steepest descent applied, $P$ is the number of minima; $A_p$ ($p=1,...,P$) are randomly chosen rotation matrices of size $d\times d$; $\Sigma_p$ ($p=1,...,P$) are diagonal positive definite matrices of size $d\times d$;  $x_{0p}$ ($p=1,...,P$) are random points in $\mathfrak{X}$. 

Note that steepest descent iterations are terminated at the smallest $k=K_n$ such that $\nabla f(x_n^{(k)}) < \delta$. 

We can find:
\begin{equation}
p_n = {\rm argmin}_{1\le p \le P}  (x_n^{(k)}-x_{0p})^T A_p^T \Sigma_p A_p (x_n^{(k)}-x_{0p})
\end{equation}

Then for $x_n^{(k)}$, the gradient is:
\begin{equation}
	\label{eq:modelgrad}
	\nabla f(x_n^{(k)})= 2  A_{p_n}^T \Sigma_{p_n} A_{p_n} (x_n^{(k)}-x_{0p_n})\,
\end{equation}




In [2]:
f = mtv3.quad_function
g = mtv3.quad_gradient

# 3. Defining parameters

The following parameters are required in order to derive $A_p$ ($p=1,...,P$) ; $\Sigma_p$ ($p=1,...,P$) and $x_{0p}$ ($p=1,...,P$).

We have:

•d: dimension

•P: number of minima

•lambda_1: smallest eigenvalue of $\Sigma_p$ ($p=1,...,P$)

•lambda_1: largest eigenvalue of $\Sigma_p$ ($p=1,...,P$)


In [3]:
d = 20
P = 5
lambda_1 = 1
lambda_2 = 10

In [4]:
store_x0, matrix_combined = mtv3.function_parameters_quad(P, d, lambda_1, lambda_2)

We have:

•store_x0: $x_{0p}$ ($p=1,...,P$)

•matrix_combined: $A_p^T \Sigma_p A_p$ ($p=1,...,P$)


In [5]:
args = P, store_x0, matrix_combined

# 4. Run METOD Algorithm

Please see ??? for more information on how to choose appropiate parameters for the METOD Algorithm. 

In [6]:
discovered_minima, number_minima, func_vals_of_minima, excessive_no_descents  = mtv3.metod(f, g, args, d)

100%|██████████| 999/999 [00:17<00:00, 55.97it/s]


# 5. Results of the METOD Algorithm

Total number of minima found:

In [7]:
number_minima

5

Positions of minima:

In [8]:
discovered_minima

[array([0.48571686, 0.54532348, 0.22858454, 0.80948386, 0.73597094,
        0.53052243, 0.16251004, 0.02229008, 0.6980381 , 0.93060659,
        0.12201367, 0.80763152, 0.42051281, 0.05591158, 0.64089777,
        0.13401022, 0.40163732, 0.41242032, 0.01281469, 0.18989694]),
 array([1.83098195e-01, 8.80944259e-01, 7.32216073e-01, 1.35802327e-01,
        9.81224478e-01, 2.01586543e-01, 7.97188978e-01, 4.53558897e-01,
        6.46596308e-01, 1.26334701e-01, 7.97751469e-01, 6.94827040e-01,
        7.26584749e-01, 4.31101045e-01, 3.30969951e-02, 9.63004172e-01,
        2.41602918e-01, 1.56868953e-01, 5.53110047e-04, 1.64174248e-01]),
 array([0.55981783, 0.56438381, 0.80250847, 0.816774  , 0.2351211 ,
        0.05794978, 0.780737  , 0.89566251, 0.06717824, 0.80418393,
        0.69675874, 0.18417236, 0.11432095, 0.60613755, 0.18863907,
        0.97941414, 0.69218386, 0.27183858, 0.59987028, 0.64599616]),
 array([0.53769703, 0.5392158 , 0.37252267, 0.3951745 , 0.54461389,
        0.0830346 , 0.

Function values of minima:

In [9]:
func_vals_of_minima

[1.4673236911527896e-11,
 1.0867771969051146e-11,
 1.1664952516620297e-11,
 1.255658033536304e-11,
 1.0777104326605999e-11]

Total number of extra descents to an already discovered minima:

In [10]:
excessive_no_descents

0

# 6. Save results to csv file (optional)

The below csv files will be saved to the same folder which contains the METOD Algorithm - Minimum of several quadratic forms notebook.

Minima in msqf_discovered_minimas_d_%s_p_%s.csv are saved such that each row represents one discovered minima. The total number of rows will be the same as the value for number_minima.

In [ ]:
np.savetxt('msqf_discovered_minimas_d_%s_p_%s.csv' % (d, P), discovered_minima, delimiter=",")

Values in msqf_func_vals_discovered_minimas_d_%s_p_%s are saved such that each row represents the function value of a minima. The total number of rows will be the same as the value for number_minima.

In [ ]:
np.savetxt('msqf_func_vals_discovered_minimas_d_%s_p_%s.csv' % (d, P), func_vals_of_minima, delimiter=",")

msqf_summary_table_d_%s_p_%s.csv will contain the total number of minima discovered and the total number of extra descents.

In [ ]:
summary_table = pd.DataFrame({
"Total number of unique minima": [number_minima],
"Extra descents": [excessive_no_descents]})
summary_table.to_csv('msqf_summary_table_d_%s_p_%s.csv' % (d, P))

# 7. Test results (optional)

This test can only be used for the minimum of several quadratic forms function.

To check each discovered minima is unique, for each minima $x_l^{(K_l)}$ found where $l=1,...,L$, we do the following:

\begin{equation}
p_l = {\rm argmin}_{1\le p \le P}  (x_l^{(K_l)}-x_{0p})^T A_p^T \Sigma_p A_p (x_l^{(K_l)}-x_{0p})
\end{equation}

In order for discovered minima to be unique, we must have that all $p_l$ is different for 
$l=1,...,L$. 

We also test that for each discovered minima $x_l^{(K_l)}$:

$$\|x_l^{(K_l)} - x_{0p_l}\| \text{  is small}$$


In [11]:
def calc_pos(point, d,  p, store_x0, matrix_combined):
    """ Checks position of where minimum of function is for Quadratic""" 
    function_values = np.zeros((p))
    for i in range(p):
        function_values[i] = np.transpose(point - store_x0[i]) @ matrix_combined[i] @ (point - store_x0[i])
    position_minimum = np.argmin(function_values)
    return position_minimum 

In [12]:
"""Store values from calc_pos function""" 
norms_with_minima = np.zeros((number_minima))
pos_list = np.zeros((number_minima))
for j in range(number_minima):
    pos = calc_pos(discovered_minima[j], d, *args)
    pos_list[j] = pos
    norms_with_minima[j] = LA.norm(discovered_minima[j]- store_x0[pos])

${\max}_{1\le l \le L}  \|x_l^{(K_l)}-x_{0p_l}\|$ should be small

In [13]:
np.max(norms_with_minima)

3.6934595421738563e-06

Ensure that the number of unique minima is $L$

In [14]:
np.unique(pos_list).shape[0] == number_minima

True